In [ ]:

import pyodbc
import os
from dotenv import load_dotenv
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import AzureOpenAI
load_dotenv()

In [ ]:
# connect to the Azure SQL database

from sqlalchemy import create_engine

connectionString=os.environ["py-connectionString"]

db_engine = create_engine(connectionString)

db = SQLDatabase(db_engine, view_support=True, schema="dbo")

# test the connection
print(db.dialect)
print(db.get_usable_table_names())
db.run("select convert(varchar(25), getdate(), 120)")


In [ ]:
azurellm = AzureOpenAI(
    deployment_name="langtrain",
    model_name="gpt-3.5-turbo-instruct",
    api_version="2023-07-01-preview",
    azure_endpoint="https://testwed.openai.azure.com",
    api_key="05c28289fa3d42daa7fa13439cb02f5e"
)

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=azurellm)
agent_executor = create_sql_agent(
    llm=azurellm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)


In [ ]:
agent_executor.invoke("count the rows in the testtable table.")